In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
df=pd.read_csv('new_file.csv')
df.head()

,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_closed_L6M,pct_closed_tl,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std,num_std_6mts,num_sub,num_sub_12mts,num_dbt,num_lss,recent_level_of_deliq,time_since_recent_enq,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_opened_TLs_L6m_of_L12m,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0,0,0.0,0.800,0.00,0.0,0,0,0,1,0,4,0,72,18,549,11,29,0,0,21,5,0,0,0,0,29,566,51000,114,0.0,0,1,0.0,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0,0,0.0,0.000,1.00,0.0,0,0,1,0,0,0,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,209,19000,50,0.0,0,0,0.0,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,1,0,0.0,0.000,0.25,0.0,1,0,6,1,0,0,0,47,2,302,9,25,8,0,10,5,0,0,0,0,25,587,18,191,0.5,0,0,0.0,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,1,0,0.0,0.000,1.00,0.0,1,0,0,0,0,0,1,5,5,-99999,0,0,0,0,5,4,0,0,0,0,0,-99999,10000,246,1.0,0,0,0.0,0.0,0,0,Married,SSC,M,others,others,P2
4,0,0,0.0,0.667,0.00,0.0,0,0,0,0,0,0,2,131,32,583,0,0,0,0,53,4,0,0,0,0,0,3951,15000,75,0.0,0,0,0.0,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1


In [4]:
df.shape

(51296, 43)

In [5]:
df.replace(-99999,np.nan,inplace=True)

In [6]:
df.dropna(inplace=True)

### Separating the dependent and Indepandent Feature

In [7]:
x=df.drop(columns='Approved_Flag')
y=df['Approved_Flag']

In [8]:
x

,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_closed_L6M,pct_closed_tl,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std,num_std_6mts,num_sub,num_sub_12mts,num_dbt,num_lss,recent_level_of_deliq,time_since_recent_enq,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_opened_TLs_L6m_of_L12m,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2
0,0,0,0.00,0.800,0.000,0.000,0,0,0,1,0,4,0,72,18,549.0,11,29,0,0,21,5,0,0,0,0,29,566.0,51000,114,0.0,0,1,0.000,0.0,1,0,Married,12TH,M,PL,PL
1,0,0,0.00,0.000,1.000,0.000,0,0,1,0,0,0,0,7,7,47.0,0,0,0,0,0,0,0,0,0,0,0,209.0,19000,50,0.0,0,0,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan
2,1,0,0.00,0.000,0.250,0.000,1,0,6,1,0,0,0,47,2,302.0,9,25,8,0,10,5,0,0,0,0,25,587.0,18,191,0.5,0,0,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others
4,0,0,0.00,0.667,0.000,0.000,0,0,0,0,0,0,2,131,32,583.0,0,0,0,0,53,4,0,0,0,0,0,3951.0,15000,75,0.0,0,0,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL
5,0,0,0.00,0.833,0.000,0.167,0,0,0,2,0,0,0,150,17,245.0,14,270,0,11,5,0,3,1,0,0,26,7.0,0,154,0.0,0,0,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51291,1,0,0.00,0.000,0.333,0.000,0,0,2,0,0,0,1,24,5,15.0,2,24,0,0,0,0,0,0,0,0,24,0.0,18500,249,1.0,0,0,0.000,0.0,0,0,Married,12TH,M,ConsumerLoan,ConsumerLoan
51292,0,1,0.25,0.500,0.500,0.250,0,0,2,0,0,0,0,74,7,57.0,0,0,0,0,6,4,0,0,0,0,0,203.0,25000,186,0.0,0,0,0.000,0.0,0,0,Married,SSC,M,others,others
51293,1,1,0.50,0.500,1.000,0.500,0,0,2,0,0,0,0,9,5,32.0,0,0,0,0,0,0,0,0,0,0,0,1.0,18000,66,0.5,0,0,1.000,0.0,0,0,Married,SSC,M,ConsumerLoan,others
51294,0,0,0.00,0.500,0.500,0.500,0,0,2,0,0,0,0,15,8,58.0,0,0,0,0,0,0,0,0,0,0,0,242.0,12802,54,0.0,0,0,0.000,0.0,0,0,Single,UNDER GRADUATE,F,ConsumerLoan,others


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
x_train.shape

(33679, 42)

In [11]:
y_train

18919    P2
39930    P3
14315    P2
50354    P2
34974    P2
         ..
7546     P2
13689    P4
46501    P2
1042     P3
19211    P2
Name: Approved_Flag, Length: 33679, dtype: object

## ML Model without OverSampling

### Handle Categorical Features

In [12]:
from sklearn.compose import ColumnTransformer

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [14]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', OneHotEncoder(sparse_output=False, drop='first'), ['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']),
    ('tnf2', OrdinalEncoder(categories=[['SSC', '12TH', 'UNDER GRADUATE', 'GRADUATE', 'POST-GRADUATE', 'PROFESSIONAL', 'OTHERS']]), ['EDUCATION'])
], remainder='passthrough')

In [15]:
x_train=transformer.fit_transform(x_train)

In [16]:
column_names = transformer.get_feature_names_out()

In [17]:
column_names

array(['tnf1__MARITALSTATUS_Single', 'tnf1__GENDER_M',
       'tnf1__last_prod_enq2_CC', 'tnf1__last_prod_enq2_ConsumerLoan',
       'tnf1__last_prod_enq2_HL', 'tnf1__last_prod_enq2_PL',
       'tnf1__last_prod_enq2_others', 'tnf1__first_prod_enq2_CC',
       'tnf1__first_prod_enq2_ConsumerLoan', 'tnf1__first_prod_enq2_HL',
       'tnf1__first_prod_enq2_PL', 'tnf1__first_prod_enq2_others',
       'tnf2__EDUCATION', 'remainder__Total_TL_opened_L6M',
       'remainder__Tot_TL_closed_L6M', 'remainder__pct_tl_closed_L6M',
       'remainder__pct_closed_tl', 'remainder__pct_tl_open_L12M',
       'remainder__pct_tl_closed_L12M', 'remainder__Tot_Missed_Pmnt',
       'remainder__CC_TL', 'remainder__Consumer_TL', 'remainder__Gold_TL',
       'remainder__Home_TL', 'remainder__PL_TL', 'remainder__Other_TL',
       'remainder__Age_Oldest_TL', 'remainder__Age_Newest_TL',
       'remainder__time_since_recent_payment',
       'remainder__num_times_delinquent',
       'remainder__max_recent_level_of_de

In [18]:
x_train = pd.DataFrame(x_train, columns=column_names)

In [19]:
x_train.head()

,tnf1__MARITALSTATUS_Single,tnf1__GENDER_M,tnf1__last_prod_enq2_CC,tnf1__last_prod_enq2_ConsumerLoan,tnf1__last_prod_enq2_HL,tnf1__last_prod_enq2_PL,tnf1__last_prod_enq2_others,tnf1__first_prod_enq2_CC,tnf1__first_prod_enq2_ConsumerLoan,tnf1__first_prod_enq2_HL,tnf1__first_prod_enq2_PL,tnf1__first_prod_enq2_others,tnf2__EDUCATION,remainder__Total_TL_opened_L6M,remainder__Tot_TL_closed_L6M,remainder__pct_tl_closed_L6M,remainder__pct_closed_tl,remainder__pct_tl_open_L12M,remainder__pct_tl_closed_L12M,remainder__Tot_Missed_Pmnt,remainder__CC_TL,remainder__Consumer_TL,remainder__Gold_TL,remainder__Home_TL,remainder__PL_TL,remainder__Other_TL,remainder__Age_Oldest_TL,remainder__Age_Newest_TL,remainder__time_since_recent_payment,remainder__num_times_delinquent,remainder__max_recent_level_of_deliq,remainder__num_deliq_6_12mts,remainder__num_times_60p_dpd,remainder__num_std,remainder__num_std_6mts,remainder__num_sub,remainder__num_sub_12mts,remainder__num_dbt,remainder__num_lss,remainder__recent_level_of_deliq,remainder__time_since_recent_enq,remainder__NETMONTHLYINCOME,remainder__Time_With_Curr_Empr,remainder__pct_opened_TLs_L6m_of_L12m,remainder__CC_Flag,remainder__PL_Flag,remainder__pct_PL_enq_L6m_of_ever,remainder__pct_CC_enq_L6m_of_ever,remainder__HL_Flag,remainder__GL_Flag
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.000,0.200,0.600,0.200,3.0,0.0,2.0,0.0,0.0,0.0,3.0,14.0,7.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.0,43000.0,122.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.333,0.333,0.333,0.333,1.0,0.0,1.0,1.0,0.0,0.0,0.0,24.0,4.0,51.0,2.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,25000.0,190.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.111,0.556,0.333,0.222,3.0,0.0,0.0,0.0,0.0,0.0,5.0,89.0,2.0,33.0,35.0,34.0,10.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,21.0,62.0,17845.0,42.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.000,0.750,0.000,0.250,0.0,0.0,2.0,0.0,0.0,0.0,1.0,89.0,17.0,41.0,0.0,0.0,0.0,0.0,28.0,3.0,0.0,0.0,0.0,0.0,0.0,75.0,12000.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.000,0.000,1.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,5.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,171.0,30000.0,125.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
x_train.shape

(33679, 50)

In [21]:
x_test=transformer.transform(x_test)

column_names = transformer.get_feature_names_out()
column_names

array(['tnf1__MARITALSTATUS_Single', 'tnf1__GENDER_M',
       'tnf1__last_prod_enq2_CC', 'tnf1__last_prod_enq2_ConsumerLoan',
       'tnf1__last_prod_enq2_HL', 'tnf1__last_prod_enq2_PL',
       'tnf1__last_prod_enq2_others', 'tnf1__first_prod_enq2_CC',
       'tnf1__first_prod_enq2_ConsumerLoan', 'tnf1__first_prod_enq2_HL',
       'tnf1__first_prod_enq2_PL', 'tnf1__first_prod_enq2_others',
       'tnf2__EDUCATION', 'remainder__Total_TL_opened_L6M',
       'remainder__Tot_TL_closed_L6M', 'remainder__pct_tl_closed_L6M',
       'remainder__pct_closed_tl', 'remainder__pct_tl_open_L12M',
       'remainder__pct_tl_closed_L12M', 'remainder__Tot_Missed_Pmnt',
       'remainder__CC_TL', 'remainder__Consumer_TL', 'remainder__Gold_TL',
       'remainder__Home_TL', 'remainder__PL_TL', 'remainder__Other_TL',
       'remainder__Age_Oldest_TL', 'remainder__Age_Newest_TL',
       'remainder__time_since_recent_payment',
       'remainder__num_times_delinquent',
       'remainder__max_recent_level_of_de

In [22]:
x_test = pd.DataFrame(x_test, columns=column_names)

In [23]:
x_test

,tnf1__MARITALSTATUS_Single,tnf1__GENDER_M,tnf1__last_prod_enq2_CC,tnf1__last_prod_enq2_ConsumerLoan,tnf1__last_prod_enq2_HL,tnf1__last_prod_enq2_PL,tnf1__last_prod_enq2_others,tnf1__first_prod_enq2_CC,tnf1__first_prod_enq2_ConsumerLoan,tnf1__first_prod_enq2_HL,tnf1__first_prod_enq2_PL,tnf1__first_prod_enq2_others,tnf2__EDUCATION,remainder__Total_TL_opened_L6M,remainder__Tot_TL_closed_L6M,remainder__pct_tl_closed_L6M,remainder__pct_closed_tl,remainder__pct_tl_open_L12M,remainder__pct_tl_closed_L12M,remainder__Tot_Missed_Pmnt,remainder__CC_TL,remainder__Consumer_TL,remainder__Gold_TL,remainder__Home_TL,remainder__PL_TL,remainder__Other_TL,remainder__Age_Oldest_TL,remainder__Age_Newest_TL,remainder__time_since_recent_payment,remainder__num_times_delinquent,remainder__max_recent_level_of_deliq,remainder__num_deliq_6_12mts,remainder__num_times_60p_dpd,remainder__num_std,remainder__num_std_6mts,remainder__num_sub,remainder__num_sub_12mts,remainder__num_dbt,remainder__num_lss,remainder__recent_level_of_deliq,remainder__time_since_recent_enq,remainder__NETMONTHLYINCOME,remainder__Time_With_Curr_Empr,remainder__pct_opened_TLs_L6m_of_L12m,remainder__CC_Flag,remainder__PL_Flag,remainder__pct_PL_enq_L6m_of_ever,remainder__pct_CC_enq_L6m_of_ever,remainder__HL_Flag,remainder__GL_Flag
0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,0.200,0.400,0.200,0.200,0.0,0.0,0.0,2.0,0.0,0.0,0.0,41.0,9.0,36.0,1.0,67.0,0.0,1.0,15.0,5.0,0.0,0.0,0.0,0.0,67.0,4.0,18500.0,38.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.500,0.500,0.000,0.500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.0,14.0,130.0,1.0,25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,445.0,23000.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000,0.400,0.200,0.000,0.0,0.0,1.0,0.0,0.0,0.0,2.0,106.0,6.0,33.0,4.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,180.0,40000.0,126.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000,0.667,0.167,0.333,0.0,0.0,6.0,0.0,0.0,0.0,0.0,37.0,3.0,71.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,4.0,19000.0,53.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.000,0.000,1.000,0.000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,35000.0,439.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8415,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.250,0.250,0.250,0.250,0.0,0.0,0.0,0.0,0.0,0.0,2.0,77.0,10.0,71.0,2.0,40.0,1.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,40.0,56.0,45000.0,185.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8416,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.333,0.667,0.667,0.667,1.0,0.0,3.0,0.0,0.0,0.0,0.0,17.0,0.0,22.0,1.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,10.0,16000.0,76.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
8417,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.182,0.727,0.182,0.182,1.0,0.0,6.0,3.0,0.0,0.0,2.0,51.0,6.0,76.0,1.0,89.0,0.0,1.0,33.0,5.0,0.0,0.0,0.0,0.0,89.0,74.0,15000.0,127.0,0.5,0.0,0.0,0.0,0.0,1.0,0.0
8418,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.000,0.000,1.000,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.0,7.0,58.0,1.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,221.0,7000.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train= le.fit_transform(y_train)

In [25]:
y_train

array([1, 2, 1, ..., 1, 2, 1], shape=(33679,))

In [26]:
y_train = pd.Series(y_train, name='Approved Flag')
y_train

0        1
1        2
2        1
3        1
4        1
        ..
33674    1
33675    3
33676    1
33677    2
33678    1
Name: Approved Flag, Length: 33679, dtype: int64

In [27]:
y_test=le.transform(y_test)

In [28]:
y_test = pd.Series(y_test, name='Approved Flag')
y_test

0       1
1       1
2       1
3       3
4       1
       ..
8415    1
8416    3
8417    1
8418    1
8419    1
Name: Approved Flag, Length: 8420, dtype: int64

# Machine Learing model fitting

### 1. Random Forest

In [29]:
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)

In [30]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


Accuracy: 0.7363420427553444



In [31]:
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.8043981481481481
Recall: 0.7027300303336703
F1 Score: 0.7501349163518618

Class p2:
Precision: 0.7728251864125932
Recall: 0.9179295414288526
F1 Score: 0.8391507736595898

Class p3:
Precision: 0.42337164750957856
Recall: 0.16986933128362797
F1 Score: 0.24245748765770708

Class p4:
Precision: 0.6206206206206206
Recall: 0.5910390848427073
F1 Score: 0.60546875



### 2. xgboost

In [32]:
!pip install xgboost


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\dell\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [33]:
import xgboost as xgb

In [34]:
xgb_classifier = xgb.XGBClassifier(
    n_estimators=373,
    max_depth=6,
    learning_rate=0.04133658965819754,
    subsample=0.7480110448735912,
    colsample_bytree=0.7655173955223075,
    gamma=0.9520437361431409,
    min_child_weight=4,
    objective='multi:softmax',  # or 'multi:softprob' if you want probabilities
    num_class=len(set(y_train)),  # total number of classes
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)

In [35]:
xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

C:\Users\dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [16:54:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [36]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()


Accuracy: 0.74



In [37]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.7997847147470398
Recall: 0.7512639029322548
F1 Score: 0.7747653806047967

Class p2:
Precision: 0.7891237640640982
Recall: 0.9110411336351112
F1 Score: 0.8457111537407509

Class p3:
Precision: 0.41651376146788993
Recall: 0.17448116833205227
F1 Score: 0.24593716143011918

Class p4:
Precision: 0.6129629629629629
Recall: 0.6310772163965681
F1 Score: 0.6218882104274307



### 3. Decision Tree

In [38]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

In [40]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()



Accuracy: 0.67



In [41]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.6752136752136753
Recall: 0.7189079878665319
F1 Score: 0.6963761018609207

Class p2:
Precision: 0.783256880733945
Recall: 0.8065341468214918
F1 Score: 0.7947251042373703

Class p3:
Precision: 0.28413910093299405
Recall: 0.25749423520368947
F1 Score: 0.2701612903225806

Class p4:
Precision: 0.5209205020920502
Recall: 0.4747378455672069
F1 Score: 0.4967581047381546



#### xgboost is giving me best results
#### We will further finetune it

# ML Model with OverSampling

In [42]:
from imblearn.over_sampling import SMOTENC

# Assuming you already have X_train and y_train
# Identify which features are categorical (even though they're numeric-encoded)
categorical_columns = [
    'tnf1__MARITALSTATUS_Single', 'tnf1__GENDER_M', 'tnf1__last_prod_enq2_CC', 
    'tnf1__last_prod_enq2_ConsumerLoan', 'tnf1__last_prod_enq2_HL', 
    'tnf1__last_prod_enq2_PL', 'tnf1__last_prod_enq2_others', 
    'tnf1__first_prod_enq2_CC', 'tnf1__first_prod_enq2_ConsumerLoan', 
    'tnf1__first_prod_enq2_HL', 'tnf1__first_prod_enq2_PL', 
    'tnf1__first_prod_enq2_others', 'tnf2__EDUCATION'
]
  # Example, update based on your data
categorical_indices = [x_train.columns.get_loc(col) for col in categorical_columns]

# Initialize SMOTENC with those categorical indices
smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42)

# Fit and resample
X_resampled, y_resampled = smote_nc.fit_resample(x_train, y_train)

In [43]:
y_resampled.value_counts()

Approved Flag
1    20393
2    20393
3    20393
0    20393
Name: count, dtype: int64

In [44]:
X_resampled

,tnf1__MARITALSTATUS_Single,tnf1__GENDER_M,tnf1__last_prod_enq2_CC,tnf1__last_prod_enq2_ConsumerLoan,tnf1__last_prod_enq2_HL,tnf1__last_prod_enq2_PL,tnf1__last_prod_enq2_others,tnf1__first_prod_enq2_CC,tnf1__first_prod_enq2_ConsumerLoan,tnf1__first_prod_enq2_HL,tnf1__first_prod_enq2_PL,tnf1__first_prod_enq2_others,tnf2__EDUCATION,remainder__Total_TL_opened_L6M,remainder__Tot_TL_closed_L6M,remainder__pct_tl_closed_L6M,remainder__pct_closed_tl,remainder__pct_tl_open_L12M,remainder__pct_tl_closed_L12M,remainder__Tot_Missed_Pmnt,remainder__CC_TL,remainder__Consumer_TL,remainder__Gold_TL,remainder__Home_TL,remainder__PL_TL,remainder__Other_TL,remainder__Age_Oldest_TL,remainder__Age_Newest_TL,remainder__time_since_recent_payment,remainder__num_times_delinquent,remainder__max_recent_level_of_deliq,remainder__num_deliq_6_12mts,remainder__num_times_60p_dpd,remainder__num_std,remainder__num_std_6mts,remainder__num_sub,remainder__num_sub_12mts,remainder__num_dbt,remainder__num_lss,remainder__recent_level_of_deliq,remainder__time_since_recent_enq,remainder__NETMONTHLYINCOME,remainder__Time_With_Curr_Empr,remainder__pct_opened_TLs_L6m_of_L12m,remainder__CC_Flag,remainder__PL_Flag,remainder__pct_PL_enq_L6m_of_ever,remainder__pct_CC_enq_L6m_of_ever,remainder__HL_Flag,remainder__GL_Flag
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.000000,0.000000,0.000000,0.200000,0.600000,0.200000,3.000000,0.0,2.000000,0.000000,0.0,0.000000,3.000000,14.000000,7.000000,88.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,98.000000,43000.000000,122.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,1.000000,0.333000,0.333000,0.333000,0.333000,1.000000,0.0,1.000000,1.000000,0.0,0.000000,0.000000,24.000000,4.000000,51.000000,2.000000,5.000000,2.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,5.000000,1.000000,25000.000000,190.000000,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.000000,1.000000,0.111000,0.556000,0.333000,0.222000,3.000000,0.0,0.000000,0.000000,0.0,0.000000,5.000000,89.000000,2.000000,33.000000,35.000000,34.000000,10.000000,0.0,9.000000,0.000000,0.000000,0.0,0.0,0.0,21.000000,62.000000,17845.000000,42.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.000000,0.000000,0.000000,0.750000,0.000000,0.250000,0.000000,0.0,2.000000,0.000000,0.0,0.000000,1.000000,89.000000,17.000000,41.000000,0.000000,0.000000,0.000000,0.0,28.000000,3.000000,0.000000,0.0,0.0,0.0,0.000000,75.000000,12000.000000,60.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1.000000,6.000000,5.000000,53.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,171.000000,30000.000000,125.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81567,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.735808,0.000000,0.471616,0.000000,0.0,1.471616,1.585151,0.0,0.000000,0.471616,38.340604,15.471616,171.807886,3.358082,18.489094,2.358082,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,18.489094,14.039289,28000.000000,66.681208,0.000000,0.0,0.000000,0.421240,0.000000,0.528384,0.0
81568,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.454113,0.969409,0.193882,0.193882,1.000000,0.193882,1.454113,0.0,1.000000,0.484704,0.0,1.454113,0.000000,5.969409,2.576478,87.995247,0.000000,0.000000,0.000000,0.0,3.877636,2.908227,0.000000,0.0,0.0,0.0,0.000000,13.790615,35000.000000,28.484704,0.903059,0.0,0.484704,0.878824,0.484704,0.484704,0.0


In [45]:
xgb_classifier = xgb.XGBClassifier(
    n_estimators=373,
    max_depth=6,
    learning_rate=0.04133658965819754,
    subsample=0.7480110448735912,
    colsample_bytree=0.7655173955223075,
    gamma=0.9520437361431409,
    min_child_weight=4,
    objective='multi:softmax',  # or 'multi:softprob' if you want probabilities
    num_class=len(set(y_resampled)),  # total number of classes
    eval_metric='mlogloss',
    random_state=42
)

In [46]:
xgb_classifier.fit(X_resampled,y_resampled)
y_pred = xgb_classifier.predict(x_test)

In [47]:
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()


Accuracy: 0.74



In [48]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.7739043824701195
Recall: 0.7856420626895855
F1 Score: 0.7797290516808831

Class p2:
Precision: 0.8011727078891258
Recall: 0.8874237354851408
F1 Score: 0.8420954337473153

Class p3:
Precision: 0.39514978601997147
Recall: 0.212913143735588
F1 Score: 0.27672327672327673

Class p4:
Precision: 0.6182152713891444
Recall: 0.6406101048617732
F1 Score: 0.6292134831460674



## Save the Model

In [50]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\dell\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [52]:
import joblib  # Make sure to import joblib

# Save the trained XGBoost model
joblib.dump(xgb_classifier, "model.joblib")

['model.joblib']

In [53]:
df.columns

Index(['Total_TL_opened_L6M', 'Tot_TL_closed_L6M', 'pct_tl_closed_L6M',
       'pct_closed_tl', 'pct_tl_open_L12M', 'pct_tl_closed_L12M',
       'Tot_Missed_Pmnt', 'CC_TL', 'Consumer_TL', 'Gold_TL', 'Home_TL',
       'PL_TL', 'Other_TL', 'Age_Oldest_TL', 'Age_Newest_TL',
       'time_since_recent_payment', 'num_times_delinquent',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std', 'num_std_6mts', 'num_sub', 'num_sub_12mts', 'num_dbt',
       'num_lss', 'recent_level_of_deliq', 'time_since_recent_enq',
       'NETMONTHLYINCOME', 'Time_With_Curr_Empr', 'pct_opened_TLs_L6m_of_L12m',
       'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS',
       'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2',
       'Approved_Flag'],
      dtype='object')